# Set up

In [ ]:
import io
import pprint
import requests
import json
from datetime import timedelta
from datetime import datetime
from keycloak import KeycloakAdmin
from keycloak import KeycloakOpenID
from IPython.display import Image

APP_BASE_URL='http://localhost:8080'
APP_ADMIN_EMAIL='app-admin@everest.engineering'
APP_ADMIN_PASSWORD='ac0n3x72'

MONITORING_PASSWORD='ac0n3x72'

KEYCLOAK_BASE_URL='http://localhost:8180'
KEYCLOAK_ADMIN_EMAIL='admin@everest.engineering'
KEYCLOAK_ADMIN_PASSWORD='ac0n3x72'

pp = pprint.PrettyPrinter()

default_client = KeycloakOpenID(
    server_url=f"{KEYCLOAK_BASE_URL}",
    realm_name='default',
    client_id='default-client',
    verify=False)

monitoring_client = KeycloakOpenID(
    server_url=f"{KEYCLOAK_BASE_URL}",
    realm_name='default',
    client_id='monitoring',
    client_secret_key=MONITORING_PASSWORD,
    verify=False)

# Create an application admin

In [ ]:
keycloak_admin = KeycloakAdmin(
    server_url=f"{KEYCLOAK_BASE_URL}/",
    username=KEYCLOAK_ADMIN_EMAIL,
    password=KEYCLOAK_ADMIN_PASSWORD,
    realm_name='default',
    user_realm_name='master',
    verify=False)

In [ ]:
admin_user_id = keycloak_admin.create_user(
    {
        "username": APP_ADMIN_EMAIL,
        "email": APP_ADMIN_EMAIL,
        "enabled": True,
        "attributes": {
            "displayName": "Application admin"
        },
        "credentials": [{"value": APP_ADMIN_PASSWORD, "type": "password"}]
    }
)

pp.pprint(admin_user_id)

Assign admin role

In [ ]:
realm_roles = keycloak_admin.get_realm_roles()
admin_role = next(role for role in realm_roles if role['name'] == 'ADMIN')
keycloak_admin.assign_realm_roles(user_id=admin_user_id, roles=[admin_role])

# User uploads photos

## Simulated user self registration for Bob

In [ ]:
keycloak_admin = KeycloakAdmin(
    server_url=f"{KEYCLOAK_BASE_URL}/",
    username=KEYCLOAK_ADMIN_EMAIL,
    password=KEYCLOAK_ADMIN_PASSWORD,
    realm_name='default',
    user_realm_name='master',
    verify=False)

In [ ]:
bob_user_id = keycloak_admin.create_user(
    {
        "username": "bob@example.com",
        "email": "bob@example.com",        
        "enabled": True,
        "attributes": {
            "displayName": "Bob Example" 
        },
        "credentials": [{"value": "password-here", "type": "password"}]
    }
)

pp.pprint(bob_user_id)

## Bob logs in

In [ ]:
bob_tokens = default_client.token('bob@example.com', 'password-here')
pp.pprint(bob_tokens)

## Bob uploads some photos

In [ ]:
with open('test_photo_1.png', 'rb') as image:
    response = requests.post(f'{APP_BASE_URL}/api/photos',
        headers=
        {
            "Authorization": f"Bearer {bob_tokens['access_token']}"
        },
        files= 
        {
            'file': image
        }
    )
    print(response.status_code)

In [ ]:
with open('test_photo_2.png', 'rb') as image:
    response = requests.post(f'{APP_BASE_URL}/api/photos',
        headers=
        {
            "Authorization": f"Bearer {bob_tokens['access_token']}"
        },
        files= 
        {
            'file': image
        }
    )
    print(response.status_code)

## Bob can download his own photos

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/photos',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
).json()
bob_photo_ids = [photo['id'] for photo in response]
pp.pprint(bob_photo_ids)

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/photos/{bob_photo_ids[0]}',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
Image(data=response.content)

### Including dynamically generated thumbnails

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/photos/{bob_photo_ids[1]}/thumbnail?width=100&height=100',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
Image(data=response.content)

# Application admin sets up competitions

## Application admin login

In [ ]:
admin_tokens = default_client.token(APP_ADMIN_EMAIL, APP_ADMIN_PASSWORD)
pp.pprint(admin_tokens)

## Create a competition

In [ ]:
response = requests.post(
    f'{APP_BASE_URL}/api/competitions', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    },
    json={
        "description": "Best holiday happy snaps",
        "submissionsOpenTimestamp": datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "submissionsCloseTimestamp": (datetime.now() + timedelta(minutes=5)).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "votingEndsTimestamp": (datetime.now() + timedelta(minutes=10)).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "maxEntriesPerUser": 1
    }
)
print(response.status_code)

## Normal users can't create competitions

In [ ]:
response = requests.post(
    f'{APP_BASE_URL}/api/competitions', 
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    },
    json={
        "description": "Bob's awesome competition",
        "submissionsOpenTimestamp": datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "submissionsCloseTimestamp": (datetime.now() + timedelta(minutes=5)).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "votingEndsTimestamp": (datetime.now() + timedelta(minutes=10)).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "maxEntriesPerUser": 100
    }
)
print(response.status_code)

# Bob enters a submission

Better log in again first

In [ ]:
bob_tokens = default_client.token('bob@example.com', 'password-here')

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/competitions',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
pp.pprint(response.json())
competition_to_enter_id = response.json()[0]['id']

You can only submit within the submission window up to the maximum number of submissions. 

In [ ]:
response = requests.post(f'{APP_BASE_URL}/api/competitions/{competition_to_enter_id}/submission',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    },
    json={
        "photoId": bob_photo_ids[1],
        "submissionNotes": "Much wow holiday snap"
    }
)

print(response.status_code)
if response.status_code != 201:
    print(response.json()['message'])

# System info

## OID configuration

In [ ]:
pp.pprint(default_client.well_know())

## Application version

In [ ]:
print(requests.get(f'{APP_BASE_URL}/api/version').text)

## Monitoring

In [ ]:
monitoring_tokens = monitoring_client.token(grant_type='client_credentials')
pp.pprint(monitoring_tokens)

In [ ]:
public_health = requests.get(f'{APP_BASE_URL}/actuator/health')
pp.pprint(public_health.json())

In [ ]:
private_health = requests.get(
    f'{APP_BASE_URL}/actuator/health', 
    headers=
    {
        "Authorization": f"Bearer {monitoring_tokens['access_token']}"
    }
)
print(private_health.status_code)
pp.pprint(private_health.json())

## Available system metrics

Most metrics won't show up until later due to lazy loading.

In [ ]:
metrics = requests.get(
    f'{APP_BASE_URL}/actuator/metrics', 
    headers=
    {
        "Authorization": f"Bearer {monitoring_tokens['access_token']}"
    }
)
print(metrics.status_code)
pp.pprint(metrics.json()['names'])

## Detailed metrics

In [ ]:
request_metrics = requests.get(
    f'{APP_BASE_URL}/actuator/metrics/process.cpu.usage', 
    headers=
    {
        "Authorization": f"Bearer {monitoring_tokens['access_token']}"
    }
)
pp.pprint(request_metrics.json())

## Prometheus metrics

Most metrics won't show up until later due ot lazy loading.

In [ ]:
prometheus_metrics = requests.get(
    f'{APP_BASE_URL}/actuator/prometheus', 
    headers=
    {
        "Authorization": f"Bearer {monitoring_tokens['access_token']}"
    }
)
print(prometheus_metrics.status_code)
pp.pprint(prometheus_metrics.text)

## Replay status

In [ ]:
replay_status = requests.get(
    f'{APP_BASE_URL}/actuator/replay', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
print(replay_status.status_code)
pp.pprint(replay_status.json())

## Trigger replay

Replays when multiple nodes are involved require you to first shut down the tracking event processors so that ownership is released. The node that receives the request to replay will be the one handling it.

This can be done via the dashboard or via `axonserver-cli`.

In [ ]:
response = requests.post(
    f'{APP_BASE_URL}/actuator/replay', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    },
    json={}
)
# You are expecting a 204 NO-CONTENT response here
print(response.status_code)